In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-979b  GCE       4                                       RUNNING  us-central1-c


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
from contextlib import closing
import pdb
import hashlib

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 12 08:37 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
w2df_dict_text = {}
bucket_name = 'yoad_big_corpus' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix='Parquet_Files/')
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_num = parquetFile.count()
doc_text_pairs = parquetFile.select("text", "id").rdd

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex
from inverted_index_gcp import MultiFileReader
from inverted_index_gcp import MultiFileWriter
TUPLE_SIZE = 6

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124


def token2bucket_id(token):
    return int(_hash(token), 16) % NUM_BUCKETS


# PLACE YOUR CODE HERE
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs
        for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    final_tokens = []
    for tok in tokens:
        if tok not in all_stopwords:
            final_tokens.append(tok)
    tf_dict = Counter(final_tokens)
    returning = []
    for elem in tf_dict.items():
        returning.append((elem[0], (id, elem[1])))
    return returning


### generating tfidf score for a document with idf and max_tf normalizing
### will be used as lambda the same way we used word_count
def tfidf1(text, id,w2df):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    final_tokens = []
    max_term = 0
    for tok in tokens:
        if tok not in all_stopwords:
            final_tokens.append(tok)
    tf_dict = Counter(final_tokens)
    for key,v in tf_dict.items():
        if v > max_term:
            max_term = v
    returning = []
    for key2,v2 in tf_dict.items():
        if key2 in w2df.keys():
            tfidf = (v2/max_term)*math.log(doc_num / w2df[key2], 10)
            tfidf = int(10000*tfidf)
        #tfidf_int = round(1000*tfidf)
            returning.append((key2, (id, tfidf)))
        #returning.append((elem[0], (id, elem[1])))
    return returning

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples
    Returns:
    --------
      list of tuples
        A sorted posting list.
    '''
    finale = []
    finale = sorted(unsorted_pl, reverse=False)
    return finale


##################
def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    finale = []
    finale = postings.map(lambda x: (x[0], len(x[1])))

    return finale


##################
def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and
        offsets its posting list was written to. See `write_a_posting_list` for
        more details.
    '''

    file_location_dictionary = {}
    lama = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().map(
        lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))

    return lama

##returning a tuple of the text's length and id
def doc_length(text,id1,w2df_dict):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    d={}
    s=0
    count=0
    for token in tokens:
        if token in w2df_dict.keys():
            if token not in d:
                d[token]=1
            else:
                count+=1

    return id1,(len(d)+count)


#for normalizing the tf by the max tf size of doc, used for lambda
def get_max_tf(text,id,w2df_dict):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    final_tokens = []
    for tok in tokens:
        if tok in w2df_dict.keys():
            final_tokens.append(tok)
    #print(max(tf_dict))
    max_term = 0
    tf_dict = dict(Counter(final_tokens))
    for key,v in tf_dict.items():
        if v > max_term:
            max_term = v
    return id, max_term

### for checking purposes, reading the posting from the index
def read_posting_list(inverted, w):
    with closing(MultiFileReader()) as reader:
        locs = inverted.posting_locs[w]
        print(locs)
        b = reader.read(locs, inverted.df[w] * TUPLE_SIZE,bucket_name)
        posting_list = []
    for i in range(inverted.df[w]):
        doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
        tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
        posting_list.append((doc_id, tf))
    print(posting_list)
    return posting_list
    




In [ ]:

# word counts map
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>50)
w2df_text = calculate_df(postings_filtered_text)
w2df_dict_text = w2df_text.collectAsMap()



In [ ]:
word_counts_text1 = doc_text_pairs.flatMap(lambda x: tfidf1(x[0], x[1],w2df_dict_text ))
postings_text2 = word_counts_text1.groupByKey().mapValues(reduce_word_counts)
postings_filtered_text = postings_text2.filter(lambda x: len(x[1])>50)

In [ ]:
list_length=doc_text_pairs.map(lambda x:doc_length(x[0],x[1],w2df_dict_text))

In [ ]:
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered_text).collect()
index_const_time = time() - t_start
# test index construction time
print(index_const_time)

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_body[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict_text
doc_length_dict=list_length.collectAsMap()
### to have the option of normalizing through doc_length
inverted.DL = doc_length_dict
# write the global stats out
inverted.write_index('.', 'index_body')
#print(read_posting_list(inverted,'orange'))
# upload to gs
index_src = "index_body.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst